In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 47.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=0154d55ac42343f22fb3a085595f0ec01091585f8043a72a5141ffdc32f0dcde
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime,timedelta
import pytz

spark = SparkSession.builder\
        .appName("Test_unixTime")\
        .getOrCreate()

In [3]:
spark

In [4]:
tz_mexico = pytz.timezone('America/Mexico_City')
fecha_hoy = datetime.now(tz_mexico).strftime('%Y-%m-%d %H:%M:%S')
fecha_hoy_dt = datetime.now(tz_mexico)

In [5]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/test1.csv', header=True)

In [6]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [7]:
df.count()

6

In [8]:
df.select('name','age').show()

+---------+---+
|     name|age|
+---------+---+
|    Krish| 31|
|Sudhanshu| 30|
|    Sunny| 29|
|     Paul| 24|
|   Harsha| 21|
|  Shubham| 23|
+---------+---+



In [9]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [10]:
df.withColumn('Age',df['age'].cast('Integer')).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [11]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [12]:
print(fecha_hoy)

2022-05-22 22:07:00


In [13]:
dff = df.withColumn('Fecha', lit(fecha_hoy))
dff.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              Fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+



In [14]:
dff.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Fecha: string (nullable = false)



In [17]:
df_unix = dff.select('Name','age','Experience','Salary',unix_timestamp(col('Fecha')).alias('fecha'))
df_unix.show()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653257220|
|Sudhanshu| 30|         8| 25000|1653257220|
|    Sunny| 29|         4| 20000|1653257220|
|     Paul| 24|         3| 20000|1653257220|
|   Harsha| 21|         1| 15000|1653257220|
|  Shubham| 23|         2|18000 |1653257220|
+---------+---+----------+------+----------+



In [18]:
df_not_unix = df_unix.select('Name','age','Experience','Salary',from_unixtime(col('Fecha')).alias('fecha'))
df_not_unix.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+



In [19]:
df_convert = df_not_unix.withColumn("fecha", unix_timestamp(col('fecha')).cast('bigint'))
df_convert.show()
df_convert.printSchema()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653257220|
|Sudhanshu| 30|         8| 25000|1653257220|
|    Sunny| 29|         4| 20000|1653257220|
|     Paul| 24|         3| 20000|1653257220|
|   Harsha| 21|         1| 15000|1653257220|
|  Shubham| 23|         2|18000 |1653257220|
+---------+---+----------+------+----------+

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- fecha: long (nullable = true)



In [20]:
df_convert_2 = df_convert.withColumn('fecha', from_unixtime(col('fecha')).cast('string'))
df_convert_2.show()
df_convert_2.printSchema()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-22 22:07:00|
|Sudhanshu| 30|         8| 25000|2022-05-22 22:07:00|
|    Sunny| 29|         4| 20000|2022-05-22 22:07:00|
|     Paul| 24|         3| 20000|2022-05-22 22:07:00|
|   Harsha| 21|         1| 15000|2022-05-22 22:07:00|
|  Shubham| 23|         2|18000 |2022-05-22 22:07:00|
+---------+---+----------+------+-------------------+

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- fecha: string (nullable = true)



In [21]:
df.select(count('Name')).collect()

[Row(count(Name)=6)]